<center><img src=img/MScAI_brand.png width=70%></center>

# Complexity of Python Data Structures



The basic Python compound data structures (`list`, `set`, `dict`, `tuple`, and a couple of
  others) offer various **operations**. Often they **depend on the number of items in the data structure**, so they are not single idealised instructions.
  
They make different trade-offs between time and memory
  usage in different common cases and worst cases. Sometimes the right
  choice can make a huge difference to our efficiency.

Sources:

* https://www.ics.uci.edu/~pattis/ICS-33/lectures/complexitypython.txt
* https://wiki.python.org/moin/TimeComplexity
* Downey, *Think Python*, Appendix B 
* Downey, *Think Complexity*, Chapter 3.


### Fundamental fact

Given a **known location in memory**, we can:

* read from it or
* write to it

in **constant time**.

### Lists

```
8   [ ][ ][ ][ ][ ][ ][ ][ ]
                          ^
```

  A Python `list` is represented internally as an **array**
  (contiguous cells in memory). Accessing a single cell is immediate (no need to visit any others).
  
  Many operations involve iterating over
  the items: $O(n)$. However, it stores a pointer to the end (so `append`
  is constant-time) and remembers its length (so no need to iterate for `len`).

  A `list` is also fast for changes at the end, but slow for changes near the
  beginning: e.g. deleting the first item means that all other items
  have to "move".

* $O(1)$: append, get and set item (`[]`), `len`
* $O(n)$: insert item, delete item, iterate over, `min`, `x in L`

### Tuple

Similar to list, for the operations that are allowed.


### Double-ended queue

  A list is fast for changes at the end, but slow for changes near the
  beginning.  If modelling a true queue we would probably want to optimise for `append` (add at the end) and
  `popleft` (remove from the front).

  But Python provides a double-ended queue (`deque`) which makes both
  ends fast, thanks to a doubly-linked list implementation.

* $O(1)$: append, appendleft, pop, popleft, `len`
* $O(n)$: get, set or remove element in the middle

### Implications

Many of our algorithms depend for run-time on careful choice of data structure
* E.g. breadth-first search needs queue behaviour ("first-in, first-out"). We can emulate this behaviour using a list (append new item at one end, copy item and delete at the other), but this will be slow: a deque is much better.

### Dictionaries

Recall that a **hash function** is the crucial ingredient in implementing 
`dict`s. A hash function maps an object to an integer,
uniformly distributed over some wide range.

Recall that Python offers a general hash function `hash`, and some objects are **not hashable**.

In [1]:
print("hash(2)", hash(2))
print("hash('abc')", hash('abc'))
print("hash([5, 6, 7])", hash([5, 6, 7]))

hash(2) 2
hash('abc') 8793657958543501247


TypeError: unhashable type: 'list'

### Implementing a dictionary from scratch


  (Warning: this is for illustration/intuition: the internal implementation is
    much more complicated)

Pretend we have a language with lists, but no dictionaries. To make a dictionary, we make a **list** `L` of some small
    fixed size, e.g. 5, and fill with non-value pairs:
```python
L = [(None, None), (None, None), ... (None, None)]
```

To add a key `k` with value `v`, we calculate a "bucket" or "pigeonhole"
which tells us where to write that key-value:
```python
i = hash(k) % 5
```
and we set 
```python
L[i] = (k, v)
```

<img src="img/hash_table_modulus.png" width=60%>

To later retrieve the value for key `k`, we calculate `i` in the same way and retrieve `L[i]`: its first element is `k` and second is `v`.

### Dictionary performance
* Calculating `hash` and looking up `list` element `i` are both $O(1)$ operations! 
  * (Constant with respect to the number of elements "filled" in `L`.)
* However, a dict uses extra memory in return for this speedup:
  * It makes a list of size 10, even when the dict is empty!
* For many situations, this trade-off is well worth it.

### Hash collisions

But sometimes two keys `k1`, `k2` could get the same `i` -- called a
*hash collision*:

In [6]:
hash(2) % 5

2

In [15]:
hash('abcdefghi') % 5 # can change in different runs/Py versions

2

So we need a bit more logic to avoid over-writing an existing
key-value, or returning the wrong value, or reporting that an key is
present when it's not.



Because each element of the list contains the key, we
can use an approach which checks it. One idea is called *linear probing*:

When we are about to add a key-value pair
at location `L[i]`, and there's already a *different* key
there, just go to `L[i+1]` and see if there is an empty space there (and repeat as needed). 

(And the same thing when retrieving: probe until we find the *correct* key.)

This works as long as the list is sparsely populated, so there are few collisions and most linear probes end quickly.

But eventually, if we keep adding key-values, we'll find a lot of
collisions, so we'll spend forever linear probing, and get
bad performance. Even worse, when the dictionary gets full, there's no space for new elements!

At that point we have to **re-hash**: 

* Make a new, much larger list `L`;
* Re-calculate all the indices (`% len(L)`);
* Copy each old element to its new location in the new list.

That will take a long time, so our **worst-case time** for item addition is $O(n)$. But it happens **rarely**.

### Amortized analysis

* Typically, adding an entry to a `dict` takes $O(1)$;
* Very rarely, it takes $O(n)$;
* On average across many additions to a single `dict`, performance is practically constant-time.

Since in any higher-level algorithm using `dict` we can expect to be adding to it many times, we are justified in using the `dict` average-case instead of worst-case. This is called **amortized analysis**.


### Dictionary performance

  Thus, we have $O(1)$ average-case performance for common dict
  operations:
* $O(1)$: `x in d`, add item, overwrite item, retrieve item, delete item, `len`
* $O(n)$: iterate over items

### Sets

  We could implement a set as a dictionary with keys and dummy values!

  That is, for a set element `x` we would use the key-value pair
  `(x, x)`.

  In early Python, that's how `set` was implemented. (It's
  optimized now.)

Result: the same $O(1)$ performance for common operations:

* $O(1)$: `x in s`, add item, delete item, `len`
* $O(n)$: iterate over items

**Exercise**: which is faster: `x in L` or `x in S` or `x in d`? (`list`, `set`, or `dict`)

**Solution**: `x in S` and `x in d` are both $O(1)$ while `x in L` is $O(n)$. 

"The performance of dictionaries is one of the minor miracles of
  computer science." -- Downey, *Think Complexity*.